In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [6]:
df = pd.read_csv('results/IMP_oracle_eval_reward.csv')
print(len(df))

2385


In [ ]:
if True:
	df = df[(df['oracle_class'] == 'OffsetBiasOracle')]
	df.to_csv('results/IMP_oracle_eval_reward.csv')

In [11]:
summary = []
oracles = ['BinaryOracle', 'MutationOracle', 'Mutation1Oracle', 'SoloOracle', 'RelativeOracle', 'DiffOracle', 'ExampleOracle', 'JointOracle', 'RankOracle']
judges = {'Meta-Llama-3.1-8B-Instruct-q8_0': '8B', 'Meta-Llama-3.1-70B-Instruct-q8_0': '70B'}
indices = []
# for oracle in oracles:
# 	for judge in judges:
# 		for do_explain in [True, False]:
# 			df_oracle = df[(df['oracle_class'] == oracle) & (df['explain'] == do_explain) & (df['judge_name'] == judge)]
# 			avg = df_oracle['pred_correct'].mean()
# 			time = df_oracle['time_taken'].mean()
# 			summary.append([avg, time, avg/time])
# 			print(oracle, judges[judge], do_explain, len(df_oracle))
# 			t_f = "T" if do_explain else "F"
# 			indices.append(f"{oracle}_{judges[judge]}_{t_f}")

oracles2 = ['ArmoRMOracle', 'InternLMOracle', 'OffsetBiasOracle']
for oracle in oracles2:
	df_oracle = df[(df['oracle_class'] == oracle)]
	avg = df_oracle['pred_correct'].mean()
	time = df_oracle['time_taken'].mean()
	summary.append([avg, time, avg/time])
	print(oracle, len(df_oracle))
	indices.append(f"{oracle}")


oracle_scoring = pd.DataFrame(data=summary,index=indices, columns=['accuracy', 'time', 'value'])
oracle_scoring

ArmoRMOracle 795
InternLMOracle 795
OffsetBiasOracle 795


,accuracy,time,value
ArmoRMOracle,NaN,NaN,NaN
InternLMOracle,NaN,NaN,NaN
OffsetBiasOracle,0.693082,0.313866,2.208212


In [4]:
def indexer(value):
    mapping = {'ResponseQuality.A_BETTER': 0, 'ResponseQuality.B_BETTER': 1, 'ResponseQuality.TIE': 2}
    return mapping[value.strip()]

def create_table(df, label=None, pred=None, oracle=None, explain=None, judge=None):
	if oracle != None:
		df = df[(df['oracle_class'] == oracle)]
	if explain != None:
		df = df[(df['explain'] == explain)]
	if judge != None:
		df = df[(df['judge_name'] == judge)]

	if label == "A":
		df = df[(df['original_label'] == 'ResponseQuality.A_BETTER')]
	elif label == "B":
		df = df[(df['original_label'] == 'ResponseQuality.B_BETTER')]
	elif label == "TIE":
		df = df[(df['original_label'] == 'ResponseQuality.TIE')]

	if pred == 1:
		df = df[(df['pred_correct'] == 1)]
	elif pred == .5:
		df = df[(df['pred_correct'] == .5)]
	elif pred == 0:
		df = df[(df['pred_correct'] == 0)]

	init_data = [[0,0,0], [0,0,0], [0,0,0]]
	for index, row in df.iterrows():
		
		i = indexer(row["original_pred"])
		j = 0
		if row["oracle_class"] not in ["SoloOracle", "BinaryOracle", "Mutation1Oracle", "DiffOracle"]:
			j = indexer(row["followup_pred"])
		init_data[i][j] += 1
	return pd.DataFrame(data=init_data,index=['A', 'B', 'TIE'], columns=['A', 'B', 'TIE'])
	

create_table(df,oracle='DiffOracle', explain=False, judge="Meta-Llama-3.1-70B-Instruct-q8_0")

,A,B,TIE
A,0,0,0
B,0,0,0
TIE,0,0,0


In [5]:
def create_table2(df, label=None, pred=None, oracle=None, explain=False, judge=None):
	if oracle != None:
		df = df[(df['oracle_class'] == oracle)]
	if explain != None:
		df = df[(df['explain'] == explain)]
	if judge != None:
		df = df[(df['judge_name'] == judge)]

	if label == "A":
		df = df[(df['original_label'] == 'ResponseQuality.A_BETTER')]
	elif label == "B":
		df = df[(df['original_label'] == 'ResponseQuality.B_BETTER')]
	elif label == "TIE":
		df = df[(df['original_label'] == 'ResponseQuality.TIE')]

	if pred == 1:
		df = df[(df['pred_correct'] == 1)]
	elif pred == .5:
		df = df[(df['pred_correct'] == .5)]
	elif pred == 0:
		df = df[(df['pred_correct'] == 0)]

	init_data = [[0,0,0], [0,0,0], [0,0,0]]
	for index, row in df.iterrows():
		i = 0 if row['original_label'] == 'ResponseQuality.A_BETTER' else (1 if row['original_label'] == 'ResponseQuality.B_BETTER' else 2)
		j = int((row["pred_correct"])*2)
		init_data[i][j] += 1
	return pd.DataFrame(data=init_data,index=['A', 'B', 'TIE'], columns=['0', '.5', '1'])
	
	
create_table2(df,oracle='DiffOracle', explain=False, judge="Meta-Llama-3.1-70B-Instruct-q8_0")

,0,.5,1
A,0,0,0
B,0,0,0
TIE,0,0,0
